<a href="https://colab.research.google.com/github/Amarmurun0212/Diver/blob/main/TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder
from keras.datasets import mnist
tf.compat.v1.disable_eager_execution()

user defined classes

In [4]:
class SampleIterator():
    def __init__(self):
        self.X = [1,2,3,4,5]
        self.counter = 0
        self.stop = len(self.X)
    def __iter__(self):
        return self
    def __next__(self):
        if self.counter >= self.stop:
            raise StopIteration()
        x = self.X[self.counter]
        self.counter += 1
        return x

class GetMiniBatch:
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

dataset preparation

In [5]:
df = pd.read_csv("Iris.csv")
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
X = np.array(X)
y = np.array(y)
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [6]:
sample_iter = SampleIterator()
for x in sample_iter:
    print(x)

1
2
3
4
5


In [7]:
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):  
    tf.random.set_random_seed(0)  
    weights = {  
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),  
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),  
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))  
    }  
    biases = {  
        'b1': tf.Variable(tf.random_normal([n_hidden1])),  
        'b2': tf.Variable(tf.random_normal([n_hidden2])),  
        'b3': tf.Variable(tf.random_normal([n_classes]))  
    }  
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])  
    layer_1 = tf.nn.relu(layer_1)  
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])  
    layer_2 = tf.nn.relu(layer_2)  
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

logits = example_net(X)  
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))  
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)  
train_op = optimizer.minimize(loss_op)  
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))   
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))  
init = tf.global_variables_initializer()

with tf.Session() as sess:  
    sess.run(init)  
    for epoch in range(num_epochs):  
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)  
        total_loss = 0  
        total_acc = 0  
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):  
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})  
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})  
            total_loss += loss  
        total_loss /= n_samples  
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})  
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))  
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})  
    print("test_acc : {:.3f}".format(test_acc))

<ipython-input-4-02beb1e0b8ea>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)


Epoch 0, loss : 7.0241, val_loss : 67.6860, acc : 0.375
Epoch 1, loss : 3.4241, val_loss : 23.4026, acc : 0.312
Epoch 2, loss : 1.9387, val_loss : 11.6681, acc : 0.375
Epoch 3, loss : 2.0917, val_loss : 13.1400, acc : 0.312
Epoch 4, loss : 1.7685, val_loss : 17.7284, acc : 0.312
Epoch 5, loss : 1.6097, val_loss : 12.9607, acc : 0.312
Epoch 6, loss : 1.4402, val_loss : 10.0593, acc : 0.312
Epoch 7, loss : 1.3704, val_loss : 9.4797, acc : 0.312
Epoch 8, loss : 1.2536, val_loss : 9.8518, acc : 0.312
Epoch 9, loss : 1.1476, val_loss : 8.5670, acc : 0.375
Epoch 10, loss : 1.0930, val_loss : 8.0430, acc : 0.375
Epoch 11, loss : 1.0412, val_loss : 7.8791, acc : 0.375
Epoch 12, loss : 0.9804, val_loss : 7.1233, acc : 0.375
Epoch 13, loss : 0.9326, val_loss : 6.7908, acc : 0.375
Epoch 14, loss : 0.8792, val_loss : 6.2492, acc : 0.375
Epoch 15, loss : 0.8304, val_loss : 5.7680, acc : 0.375
Epoch 16, loss : 0.7835, val_loss : 5.2886, acc : 0.438
Epoch 17, loss : 0.7384, val_loss : 4.8037, acc : 0

In [8]:
print("It is much faster than implemented CNN from scratch")
print("Main structure is same.")
print("First, weights and biases are initialized and then layers are defined.")
print("Next, loop through epochs. Training data is mini batch size.")
print("So forth, loss is calculated and weights and biases are updated. Finally network is evaluated by test dataset")

It is much faster than implemented CNN from scratch
Main structure is same.
First, weights and biases are initialized and then layers are defined.
Next, loop through epochs. Training data is mini batch size.
So forth, loss is calculated and weights and biases are updated. Finally network is evaluated by test dataset


[Problem 3] Create a model of Iris using all three types of objective variables

In [9]:
df = pd.read_csv("Iris.csv")
#df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
X = np.array(X)
y = np.array(y)
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y[y == "Iris-setosa"] = 2
#y = y.astype(np.int64)[:, np.newaxis]

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y[:,np.newaxis])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

n_classes = 3
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

logits = example_net(X)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))  
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)  
train_op = optimizer.minimize(loss_op)  
correct_pred = tf.equal(tf.argmax(Y,1), tf.argmax(tf.nn.softmax(logits) ,1))   
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))  
init = tf.global_variables_initializer()


with tf.Session() as sess:  
    sess.run(init)  
    for epoch in range(num_epochs):  
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)  
        total_loss = 0  
        total_acc = 0  
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):  
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})  
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})  
            total_loss += loss  
        total_loss /= n_samples  
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})  
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))  
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})  
    print("test_acc : {:.3f}".format(test_acc))

print("Sigmoid function is used to binary classification problem.")
print("In other hands, multiclass problem we used softmax function.")
print("So three class classification problem, we replace sigmoid functions to softmax functions.")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
<ipython-input-4-02beb1e0b8ea>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default

Epoch 0, loss : 14.6136, val_loss : 53.3587, acc : 0.417
Epoch 1, loss : 8.1546, val_loss : 32.2809, acc : 0.708
Epoch 2, loss : 4.0434, val_loss : 10.6616, acc : 0.333
Epoch 3, loss : 2.2728, val_loss : 8.8245, acc : 0.417
Epoch 4, loss : 1.7518, val_loss : 6.1692, acc : 0.667
Epoch 5, loss : 1.0093, val_loss : 3.9090, acc : 0.417
Epoch 6, loss : 0.5575, val_loss : 2.0676, acc : 0.750
Epoch 7, loss : 0.1764, val_loss : 1.1131, acc : 0.792
Epoch 8, loss : 0.0543, val_loss : 1.1151, acc : 0.833
Epoch 9, loss : 0.0205, val_loss : 0.9923, acc : 0.833
Epoch 10, loss : 0.0147, val_loss : 0.9817, acc : 0.833
Epoch 11, loss : 0.0120, val_loss : 0.9507, acc : 0.833
Epoch 12, loss : 0.0116, val_loss : 0.9524, acc : 0.833
Epoch 13, loss : 0.0110, val_loss : 0.9104, acc : 0.833
Epoch 14, loss : 0.0105, val_loss : 0.9079, acc : 0.833
Epoch 15, loss : 0.0102, val_loss : 0.8847, acc : 0.833
Epoch 16, loss : 0.0098, val_loss : 0.8738, acc : 0.833
Epoch 17, loss : 0.0094, val_loss : 0.8638, acc : 0.83

[Question 4] Create a House Prices model

In [11]:
house_data = pd.read_csv('train.csv')
#print(house_data.head())

#data = house_data.loc[:,['GrLivArea', 'YearBuilt', 'SalePrice']]

X = house_data[['GrLivArea', 'YearBuilt']].to_numpy()
y = house_data[['SalePrice']].to_numpy()
print("Xshape:", X.shape)
print("yshape:", y.shape)
X = np.log1p(X)
y = np.log1p(y)

print("Xshape:", X.shape)
print("yshape:", y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=123, test_size=0.2)

learning_rate = 0.001
batch_size = 10
num_epochs = 50
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

def reg_net(x):  
    tf.random.set_random_seed(0)  
    weights = {  
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),  
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),  
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))  
    }  
    biases = {  
        'b1': tf.Variable(tf.random_normal([n_hidden1])),  
        'b2': tf.Variable(tf.random_normal([n_hidden2])),  
        'b3': tf.Variable(tf.random_normal([n_classes]))  
    }  
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])  
    layer_1 = tf.nn.relu(layer_1)  
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])  
    layer_2 = tf.nn.relu(layer_2)  
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output
    
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

logits = reg_net(X)
loss_op = tf.reduce_mean(tf.square(logits -Y))  
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)  
train_op = optimizer.minimize(loss_op)  
mean_squared_erro = tf.reduce_mean(tf.square(logits -Y)) 
init = tf.global_variables_initializer()


with tf.Session() as sess:  
    sess.run(init)  
    for epoch in range(num_epochs):  
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)  
        total_loss = 0  
        total_mse = 0  
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):  
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})  
            loss, mse = sess.run([loss_op, mean_squared_erro], feed_dict={X: mini_batch_x, Y: mini_batch_y})  
            total_loss += loss  
            total_mse += mse
        total_loss /= n_samples
        total_mse /= n_samples  
        val_loss, val_mse = sess.run([loss_op, mean_squared_erro], feed_dict={X: X_val, Y: y_val})  
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, mse : {:.3f}".format(epoch, total_loss, val_loss, mse))  
    test_mse = sess.run(mean_squared_erro, feed_dict={X: X_test, Y: y_test})  
    print("test_mse : {:.3f}".format(test_mse))

print("Classification is the task of predicting a discrete class label. \n \
     Regression is the task of predicting a continuous quantity.")
print("Calculation of loss is different. In classification we used Cross-entropy loss besides in regression we used mean square error")
print("Also, activation is not used in regression.")


Xshape: (1460, 2)
yshape: (1460, 1)
Xshape: (1460, 2)
yshape: (1460, 1)


<ipython-input-4-02beb1e0b8ea>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)


Epoch 0, loss : 42.8486, val_loss : 12.5169, mse : 116.571
Epoch 1, loss : 0.4298, val_loss : 0.8426, mse : 50.609
Epoch 2, loss : 0.0790, val_loss : 0.1548, mse : 31.929
Epoch 3, loss : 0.0404, val_loss : 0.1096, mse : 21.768
Epoch 4, loss : 0.0291, val_loss : 0.1125, mse : 14.666
Epoch 5, loss : 0.0245, val_loss : 0.1587, mse : 10.530
Epoch 6, loss : 0.0218, val_loss : 0.2418, mse : 7.675
Epoch 7, loss : 0.0192, val_loss : 0.2456, mse : 5.778
Epoch 8, loss : 0.0164, val_loss : 0.1818, mse : 4.431
Epoch 9, loss : 0.0144, val_loss : 0.1283, mse : 3.440
Epoch 10, loss : 0.0134, val_loss : 0.1026, mse : 2.676
Epoch 11, loss : 0.0127, val_loss : 0.0978, mse : 1.994
Epoch 12, loss : 0.0121, val_loss : 0.1016, mse : 1.499
Epoch 13, loss : 0.0114, val_loss : 0.1245, mse : 0.999
Epoch 14, loss : 0.0109, val_loss : 0.1654, mse : 0.694
Epoch 15, loss : 0.0104, val_loss : 0.2021, mse : 0.517
Epoch 16, loss : 0.0099, val_loss : 0.2121, mse : 0.397
Epoch 17, loss : 0.0095, val_loss : 0.2078, mse :

[Problem 5] Create an MNIST model

In [12]:
(X_train,y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1,784)
X_test = X_test.reshape(-1,784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:,np.newaxis])
y_test_one_hot = enc.fit_transform(y_test[:,np.newaxis])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2, random_state=0)

learning_rate = 0.001
batch_size = 10
num_epochs = 30
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])


get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):  
    tf.random.set_random_seed(0)  
    weights = {  
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),  
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),  
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))  
    }  
    biases = {  
        'b1': tf.Variable(tf.random_normal([n_hidden1])),  
        'b2': tf.Variable(tf.random_normal([n_hidden2])),  
        'b3': tf.Variable(tf.random_normal([n_classes]))  
    }  
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])  
    layer_1 = tf.nn.relu(layer_1)  
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])  
    layer_2 = tf.nn.relu(layer_2)  
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

logits = example_net(X)  
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))  
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)  
train_op = optimizer.minimize(loss_op)  
correct_pred = tf.equal(tf.argmax(Y,1), tf.argmax(tf.nn.softmax(logits),1))   
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))  
init = tf.global_variables_initializer()

with tf.Session() as sess:  
    sess.run(init)  
    for epoch in range(num_epochs):  
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)  
        total_loss = 0  
        total_acc = 0  
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):  
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})  
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})  
            total_loss += loss  
        total_loss /= n_samples  
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})  
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))  
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})  
    print("test_acc : {:.3f}".format(test_acc))
#Please summarize it in simple words. 

print("In this problem, input data is an image and number of classes are 10.")
print("Hence input is an image. There are many networks which is classify images such as lenet, alexnet and so on.")

11490434/11490434 [==============================] - 0s 0us/step


<ipython-input-12-7ee9a0c8d273>:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train = X_train.astype(np.float)
<ipython-input-12-7ee9a0c8d273>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test = X_test.astype(np.float)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was re

Epoch 0, loss : 2.7970, val_loss : 8.1313, acc : 0.800
Epoch 1, loss : 0.4951, val_loss : 3.6695, acc : 0.831
Epoch 2, loss : 0.2293, val_loss : 1.9933, acc : 0.829
Epoch 3, loss : 0.1244, val_loss : 1.3123, acc : 0.837
Epoch 4, loss : 0.0792, val_loss : 1.0061, acc : 0.849
Epoch 5, loss : 0.0582, val_loss : 0.8401, acc : 0.869
Epoch 6, loss : 0.0468, val_loss : 0.7956, acc : 0.881
Epoch 7, loss : 0.0393, val_loss : 0.7329, acc : 0.882
Epoch 8, loss : 0.0339, val_loss : 0.7136, acc : 0.892
Epoch 9, loss : 0.0304, val_loss : 0.6784, acc : 0.896
Epoch 10, loss : 0.0271, val_loss : 0.6662, acc : 0.902
Epoch 11, loss : 0.0247, val_loss : 0.6397, acc : 0.905
Epoch 12, loss : 0.0230, val_loss : 0.6441, acc : 0.906
Epoch 13, loss : 0.0208, val_loss : 0.6370, acc : 0.913
Epoch 14, loss : 0.0196, val_loss : 0.6205, acc : 0.913
Epoch 15, loss : 0.0187, val_loss : 0.6242, acc : 0.916
Epoch 16, loss : 0.0175, val_loss : 0.6572, acc : 0.919
Epoch 17, loss : 0.0166, val_loss : 0.6175, acc : 0.917
Ep